### 2.1 Preprocesado

Primeramente, se elimina el nombre de la ciudad y país 

-- hablar de la eliminacion de ciudades, el pais y posicion 2021 y 2022, comentando pq lo hicimos.

-- mencionar que las otras columnas que eliminaremos en los test se introduciran mas adelante.

### 2.2 Metaparametros

-- explicar los metaparametros

### 2.3 Metodo de inicializacion

### 2.4 Manejo de cluster vacios en kmeans

exlicar que nos quedamos con el de menor distancia al centroide vacio y re asignamos

## 3 Experimentos

### 3.1 


In [ ]:
from test import Test

test = Test()
dataset = [] #TODO: funcion para traer dataset
#retrieve_spam_dataset()
test.

# Clustering y PCA sobre dataset de work-life balance

## Laboratorio 2022 - Tarea 4

### Grupo 27

Alexis Baladón, CI: 5.574.612-4

Ignacio Viscardi, CI: 5.066.666-2

Rafael Castelli, CI: 5.079.391-2

## Requisitos para reproducción de algoritmo

Para correr el presente notebook se deberán instalar las siguientes dependencias. Queda un script para su instalación automática.

In [ ]:
!pip install -r requirements.txt

# Clasificación de correos por medio de naive Bayes

## Laboratorio 2022 - Tarea 3

### Grupo 16

Alexis Baladón, CI: 5.574.612-4

Ignacio Viscardi, CI: 5.066.666-2

Rafael Castelli, CI: 5.079.391-2

## 2. Decisiones de diseño

La finalidad de esta sección consiste en introducir las principales decisiones de diseño tomadas para esta implementación de K-means.

### 2.2 Metaparametros

-- explicar los metaparametros

### 2.4 Manejo de cluster vacios en kmeans

exlicar que nos quedamos con el de menor distancia al centroide vacio y re asignamos

In [ ]:
from test import Test

test = Test()
dataset = [] #TODO: funcion para traer dataset
#retrieve_spam_dataset()
test.

## 4 Conclusiones

## 4.1 Resultados según expectativas iniciales
 Joya